<a href="https://colab.research.google.com/github/sonalsrivas/Mathematical-Equation-Solver/blob/master/Up_and_Running_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Changing the rules of identifying keyword for numerical data to be
 **only dependent on its unit**

 Because of conflicts in this smaple problem: *"A train travels the first 15km at a uniform speed of 30km/h; the next 75km at a uniform speed of 50km/h. and the last 10km at a uniform speed of 20km/h. Calculate the average speed for the entire train journey."*

In [0]:
from nltk import word_tokenize as Nwt

map_unit_UNIT=dict()
map_UNIT_unit={'kmph':['kmph','km/h','km/hr'],'mps':['mps','m/s','m/sec'],'C':['celsius','centigrade','c','degree'],'F':['f','fahrenheit'],
               'hour':['hour','hr'],'minute':['min','minute'],'second':['second','sec','s'],'day':['day'],'year':['year','yr'],
               'meter':['m','meter','metre'],'kilometer':['kilometer','km'],}
for vt in map_UNIT_unit:                                                         #this will save a little time
  for ut in map_UNIT_unit[vt]:
    map_unit_UNIT[ut]=vt
#print(map_unit_UNIT)

#map_unit_UNIT={'kmph':'kmph','km/h':'kmph','km/hr':'kmph',
#              'mps':'mps','m/s':'mps','m/min':'mpm',
#              'centigrade':'C','celsius':'C','degrees':'C','fahrenheit':'F'}


map_VAR_UNIT={'speed':{'kmph','mps'},'temperature':{'C','F'}, 'time':{'hour','minute','second','day','year'},'distance':['meter','kilometer'] }
#                                                                                                               !----------------!
#DONE #you could have done this with code 
map_UNIT_VAR=dict() #{'kmph':'speed','mps':'speed','C':'temperature','F':'temperature'}
for vt in map_VAR_UNIT:                                                         #this will save a little time
  for ut in map_VAR_UNIT[vt]:
    map_UNIT_VAR[ut]=vt
#print(map_UNIT_VAR)

map_VAR_var={'speed':{'velocity','speed'},'distance':{'distance','displacement','dist.'},'temparature':{'temparature'},'time':{'time'}}
map_var_VAR=dict()
for vt in map_VAR_var:                                                         #this will save a little time
  for ut in map_VAR_var[vt]:
    map_var_VAR[ut]=vt
#print(map_var_VAR)






''' cleans problem and splits in words '''
def listproblem(string):
  #s=remove_punctuations(string)
  problem=Nwt(string)
  for p in range(len(problem)):
    if problem[p][-1]=='s' and problem[p][-2] not in 'aoui/':
      problem[p]=problem[p][:-1]
    #print("PROBLEM IN listproblem::",problem)                                                         #
  return problem

''' TransformWords insures that: numeral data is separated from alphabetic data, like -24m/s is spaced; '''
def TransformWords(s):
  l=len(s); i=0
  while i<l:
    if s[i].isdigit() and s[i+1].isalpha():
      s=s[:i+1]+' '+s[i+1:]
    i+=1
  return s

''' convert string to integer numeral or decimal numeral '''
def convert_numeral(word):
  #print(word)
  if '.' in word:
    return float(word)
  else:
    return int(word)


'''produces a list containing numerical data and its given unit in the problem. The unit is determined simply by the word next to the numerical value.'''
def numeralunitList(problem):
  now=len(problem)
  numerical_data=[]                   #Of the form (id,numeral,unit,variable(optional))
  i=0
  while i<now:
    word=problem[i]
    i+=1
    if word[0].isdigit():
      assocUnit=problem[i]
      if assocUnit[-1]=='.':
        assocUnit=assocUnit[:-1]
        problem[i]=problem[i][:-1]
        problem.insert(i+1,' ')
        now+=1
      assocUnit=map_unit_UNIT[assocUnit]
      numerical_data.append([i-1,convert_numeral(word),assocUnit])
      i+=1
  return numerical_data

def wordTOnum(s):
  rest={0:'zero',1:'one',2:'two',3:'three',4:'four',5:'five',6:'six',7:'seven',8:'eight',9:'nine'}
  tens={20:'twenty',30:'thirty',40:'forty',50:'fifty',60:'sixty',70:'seventy',80:'eighty',90:'ninety'}
  elevtens={10:'ten',11:'eleven',12:'twelve',13:'thirteen',14:'fourteen', 15:'fifteen',16:'sixteen',17:'seventeen', 18:'eighteen',19:'nineteen'}
  
  number={**{rest[i]:i for i in rest},**{tens[i]:i for i in tens},**{elevtens[i]:i for i in elevtens}}
  denomination={'hundred':100,'thousand':1000,'million':1000000,'lakh':100000}

  now=len(s); w=0
  while w<now:
    last=1; num=0
    flag=-1; first=w
    #print("w,s[w]")
    #print(w,s[w])
    while s[w] in denomination or s[w] in number:
      #print("num,w,s[w]")
      #print(num,w,s[w])
      if s[w] in denomination:
        #print("num,s[w],last*denomination[s[w]]",num,s[w],last*denomination[s[w]])
        num+=last*denomination[s[w]]; flag=1
      else:
        #print("last,number[s[w]]",last,number[s[w]])
        if last in tens:
          last+=number[s[w]]
        else:
          last=number[s[w]]
        flag=0
      #print(num)
      w+=1; now-=1
    if flag==0:
      num+=last
    if flag!=-1:
      #print("last,first,w,s[first:w],str(num)")
      #print(last,first,w,s[first:w],str(num))
      s[first:w]=[str(num)]
      now+=1
    w+=1
  #print(s)
  return s
    


def var_associations(problem,numerical_data):
  #check the words following and previuos to numeral is a keyword until 'and' or ',' or '.' appears
  ## numerical_data has structure of list with elements like (index, numeral, unit)
  ### we want another structure or will later modify numerical_data , that has (variable, numeral, unit) and maybe index as additional information
  #==> VarNumUnit=[]
  VarNumUnit=[]
  for j in numerical_data:
    l=len(problem)
    VarNumUnit.append((map_UNIT_VAR[j[2]],j[1],j[2]))
  return VarNumUnit




string=input(":Enter your word problem:\n").lower()
while string!='':
  string=TransformWords(string)
  problem=listproblem(string)
  problem=wordTOnum(problem)
  print("The problem =>",problem)
  numerical_data=numeralunitList(problem)
  #print("Index, Numeral, Unit =>",numerical_data)
  #Answer=wordproblem(problem, numerical_data)
  #print(Answer)
  #print("The problem =>",problem)
  
  Var_Num_Unit=var_associations(problem,numerical_data)
  print("\nKeyword, Numeral, Unit :\n",Var_Num_Unit)
  
  string=input("\n:Enter your word problem:\n").lower()

:Enter your word problem:
Change the speed of six hundred forty seven m/s into km/h.
The problem => ['change', 'the', 'speed', 'of', '647', 'm/s', 'into', 'km/h', '.']

Keyword, Numeral, Unit :
 [('speed', 647, 'mps')]

:Enter your word problem:
A ball hits a wall horizontally at 6m/s. It rebounds horizontally at 4.4m/s. The ball is in contact with the wall for 0.040 s. What is the acceleration of the ball?
The problem => ['a', 'ball', 'hit', 'a', 'wall', 'horizontally', 'at', '6', 'm/s', '.', 'it', 'rebound', 'horizontally', 'at', '4.4', 'm/s', '.', 'the', 'ball', 'is', 'in', 'contact', 'with', 'the', 'wall', 'for', '0.040', 's.', 'what', 'is', 'the', 'acceleration', 'of', 'the', 'ball', '?']

Keyword, Numeral, Unit :
 [('speed', 6, 'mps'), ('speed', 4.4, 'mps'), ('time', 0.04, 'second')]

:Enter your word problem:
If a sprinter runs a distance of 100 metres in 9.83 seconds, calculate his average speed in km/h.
The problem => ['if', 'a', 'sprinter', 'run', 'a', 'distance', 'of', '100'